## Demo of Astronomical Interactive Data Analysis Techniques using IPython/Jupyter Notebooks and Ginga

This notebook shows examples of using an interactive Ginga viewer running in an HTML5 canvas with an IPython Notebook.  You do not need a special widget set to run this, just an HTML5 compliant browser.

See NOTES at end for more info installation/requirements.

This is adapted for AAS 229 workshop based on the original demo [here](https://github.com/ejeschke/ginga/blob/master/ginga/examples/ipython-notebook/ginga_ipython_demo.ipynb).

In [ ]:
# Requirements:
from ginga.version import version
version

# Get ginga from github (https://github.com/ejeschke/ginga) or
#   pypi (https://pypi.python.org/pypi/ginga)
# Ginga documentation at: http://ginga.readthedocs.io/en/latest/

In [ ]:
# setup
from ginga.web.pgw import ipg
# Set this to True if you have a non-buggy python OpenCv bindings--it greatly speeds up some operations
use_opencv = False

server = ipg.make_server(host='localhost', port=9914, use_opencv=use_opencv)

In [ ]:
# Start viewer server
# IMPORTANT: if running in an IPython/Jupyter notebook, use the no_ioloop=True option
server.start(no_ioloop=True)

In [ ]:
# Get a viewer
# This will get a handle to the viewer v1 = server.get_viewer('v1')
v1 = server.get_viewer('v1')

In [ ]:
# where is my viewer
v1.url

In [ ]:
# open the viewer in a new window
v1.open()

*NOTE*: if you don't have the `webbrowser` module, *open the link that was printed in the cell above in a new window* to get the viewer.  

You can open as many of these viewers as you want--just keep a handle to it and use a different name for each unique one.

Keyboard/mouse bindings in the viewer window: http://ginga.readthedocs.io/en/latest/quickref.html

You will want to check the box that says "I'm using a trackpad" if you are--it makes zooming much smoother

In [ ]:
# Load an image into the viewer
# (change the path to where you downloaded the sample images, or use your own)
v1.load('../Lecture_Notebooks/Imexam/iacs01t4q_flt.fits')

In [ ]:
# Example of embedding a viewer
v1.embed(height=650)

In [ ]:
# capture the screen
v1.show()


In [ ]:
# Let's get the pan position we just set
dx, dy = v1.get_pan()
dx, dy

In [ ]:
# Getting values from the FITS header is also easy
img = v1.get_image()
hdr =img.get_header()
hdr['BUNIT']

In [ ]:
# What are the coordinates of the pan position?
# This uses astropy.wcs under the hood if you have it installed
img.pixtoradec(dx, dy)

In [ ]:
# Set cut level algorithm to use
v1.set_autocut_params('zscale', contrast=0.25)
# Auto cut levels on the image
v1.auto_levels()

In [ ]:
# Let's do an example of the two-way interactivity
# First, let's add a drawing canvas
canvas = v1.add_canvas()

In [ ]:
# delete all objects on the canvas
canvas.delete_all_objects()
# set the drawing parameters
canvas.set_drawtype('circle', color='blue')

Now, in the Ginga window, draw a point using the right mouse button (if you only have one mouse button (e.g. Mac) press and release spacebar, then click and drag)

In [ ]:
# get the pixel coordinates of the point we just drew
p = canvas.objects[0]
p.x, p.y

In [ ]:
# Get the RA/DEC in degrees of the point
img.pixtoradec(p.x, p.y)

In [ ]:
# Get RA/DEC in H M S sign D M S
img.pixtoradec(p.x, p.y, format='hms')

In [ ]:
# Get RA/DEC in classical string notation
img.pixtoradec(p.x, p.y, format='str')

In [ ]:
# Verify we have a valid coordinate system defined
img.wcs.coordsys

In [ ]:
# Get viewer model holding data
image = v1.get_image()
image.get_minmax()

In [ ]:
# get viewer data
data_np = image.get_data()
import numpy as np
np.mean(data_np)

In [ ]:
# Set viewer cut levels
v1.cut_levels(0, 5)

In [ ]:
# set a color map on the viewer 
v1.set_color_map('smooth')

In [ ]:
# Image will appear in this output
v1.show()

In [ ]:
# Set color distribution algorithm
# choices: linear, log, power, sqrt, squared, asinh, sinh, histeq, 
v1.set_color_algorithm('linear')

In [ ]:
# Example of setting another draw type.
canvas.delete_all_objects()
canvas.set_drawtype('rectangle', color='green')

Now right-drag to draw a *small* rectangle in the Ginga image.
Remember: On a single button pointing device, press and release spacebar, then click and drag.

Try to include some objects. 


In [ ]:
# Find approximate bright peaks in a sub-area
from ginga.util import iqcalc
iq = iqcalc.IQCalc()

img = v1.get_image()
r = canvas.objects[0]
data = img.cutout_shape(r)
peaks = iq.find_bright_peaks(data)
peaks[:20]

In [ ]:
# evaluate peaks to get FWHM, center of each peak, etc.
objs = iq.evaluate_peaks(peaks, data)
# how many did we find with standard thresholding, etc.
# see params for find_bright_peaks() and evaluate_peaks() for details
len(objs)

In [ ]:
# example of what is returned
o1 = objs[0]
o1

In [ ]:
# pixel coords are for cutout, so add back in origin of cutout
#  to get full data coords RA, DEC of first object
x1, y1, x2, y2 = r.get_llur()
img.pixtoradec(x1+o1.objx, y1+o1.objy)

In [ ]:
# Draw circles around all objects
Circle = canvas.get_draw_class('circle')
for obj in objs:
    x, y = x1+obj.objx, y1+obj.objy
    if r.contains(x, y):
        canvas.add(Circle(x, y, radius=100, color='yellow'))
        
# set pan and zoom to center
v1.set_pan((x1+x2)/2, (y1+y2)/2)
v1.scale_to(0.75, 0.75)

In [ ]:
v1.show()

How about some plots...?

In [ ]:
# swap XY, flip Y, change colormap back to "ramp"
v1.set_color_map('gray')
v1.transform(False, True, True)
v1.auto_levels()

In [ ]:
# Programmatically add a line along the figure at designated coordinates
canvas.delete_all_objects()
Line = canvas.get_draw_class('line')
l1 = Line(0, 585, 250, 585)
tag = canvas.add(l1)

In [ ]:
# Set the pan position and zoom to 1:1.  Show what we did.
v1.set_pan(125, 512)
v1.scale_to(1.0, 1.0)

In [ ]:
v1.show()

In [ ]:
# Get the pixel values along this line
img = v1.get_image()
values = img.get_pixels_on_line(l1.x1, l1.y1, l1.x2, l1.y2)
values[:10]

In [ ]:
# Plot the 'cuts'
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.cla()
plt.plot(values)
plt.ylabel('Pixel value')

plt.show()

In [ ]:
# Plot the cuts that we will draw interactively
canvas.delete_all_objects()
canvas.set_drawtype('line')

Now draw a line through the image (remember to use right mouse btn or else press space bar first)

In [ ]:
# show our line we drew
v1.show()

In [ ]:
def getplot(v1):
    l1 = canvas.objects[0]
    img = v1.get_image()
    values = img.get_pixels_on_line(l1.x1, l1.y1, l1.x2, l1.y2)
    plt.cla()
    
    plt.plot(values)
    plt.ylabel('Pixel value')
    plt.show()

In [ ]:
getplot(v1)

## Other ways of loading data

In [ ]:
# make some random data in a numpy array
import numpy as np
import random
data_np = np.random.rand(512, 512)

In [ ]:
# example of loading numpy data directly to the viewer
v1.load_data(data_np)
v1.show()

Th-th-th-that's all folks!

## Requirements

Needed packages for this notebook:

- [`ginga`](https://github.com/ejeschke/ginga), jupyter/ipython w/notebook feature
- typical scientific python reqs: `numpy`, `scipy`, `astropy`
- for drawing, you will need either PIL/pillow, OpenCv or the [`aggdraw` module](https://github.com/ejeschke/aggdraw) module (python 2 only).  PIL is included in anaconda, so is usually all you need.
- optional, but highly recommended: `webbrowser`, OpenCv

Latest Ginga documentation, including detailed installation instructions, can be found [here](http://ginga.readthedocs.io/en/latest/).